# local

In [34]:
%matplotlib tk
import numpy as np
import matplotlib.pyplot as plt
import kwant
import time
from numpy import sqrt,pi,cos,sin,kron
from scipy.sparse.linalg import eigsh

#pauli matrix
sigma_0 = np.array([[1, 0],[0, 1]])
sigma_x = np.array([[0, 1],[1, 0]])
sigma_y = np.array([[0, -1j],[1j, 0]])
sigma_z = np.array([[1, 0],[0, -1]])

tau_0 = sigma_0
tau_x = sigma_x
tau_y = sigma_y
tau_z = sigma_z


'''固定参数'''
hbar = 1.055e-34    #J.S   不确定要不要
me = 9.109e-31    #kg
meff = 0.03*me
delta0 = 0.12   #meV


a = 1e-8 #m
L = 300

#g·mu_B的值
gmu = 1.446  #meV/T
meV = 1.60217e-22
t = hbar**2/(2*meff*a**2)/meV   #量纲为meV
gamma = 0.15 #meV


'''变化参数'''
mu = 0
alpha = 0.8  #约化Rashba，等于alpha/a,单位为meV
B = 0.
mu_n = 15   #meV  
t_n = 0.3*t

#self energy
def self_eng(omega):
    return -gamma/sqrt(delta0**2-omega**2)*(omega*kron(tau_0,sigma_0) + delta0*kron(tau_x,sigma_0))


def make_sys(B,omega,mu):
    lat = kwant.lattice.square(a=1,norbs=4)
    sys = kwant.Builder()
    es = 1e-3
    def system_shape(pos):
        return 0 - es < pos[0] < L + es and 0-es < pos[1]<1
    
    def lead_left(pos):
        return 0-es < pos[1] < 1 
    def lead_right(pos):
        return 0-es < pos[1] < 1 
    
    def onsite_SC(site):
        return (2*t-mu)*kron(tau_z,sigma_0) + self_eng(omega) + 1/2*gmu*B*kron(tau_0,sigma_x) 
    
    def hoppingx(site1,site2):
        x,y = site1.pos
        return  -t*kron(tau_z,sigma_0) + 1j*alpha/2*kron(tau_z,sigma_y) 


    sys[lat.shape(system_shape, (0, 0))] = onsite_SC

    sys[kwant.builder.HoppingKind((1, 0), lat, lat)] = hoppingx

    #left lead
    left = kwant.Builder(kwant.TranslationalSymmetry((-1,0)))
    left[lat.shape(lead_left, (-L//2 , 0))] = (2*t_n - mu_n)*kron(tau_z,sigma_0)
    left[kwant.builder.HoppingKind((1, 0), lat, lat)] = -t_n *kron(tau_z,sigma_0)
    sys.attach_lead(left)

    # right lead
    right = kwant.Builder(kwant.TranslationalSymmetry((1,0)))
    right[lat.shape(lead_right, (L//2 , 0))]= (2*t_n - mu_n)*kron(tau_z,sigma_0) 
    right[kwant.builder.HoppingKind((1, 0), lat, lat)] = -t_n*kron(tau_z,sigma_0)
    sys.attach_lead(right)
    return sys

def local_cond(S):
    ree=0
    reh=0
    for i in range(len(S)//2):
        for j in range(len(S)//2):
            ree=ree+np.abs(S[i,j])**2
    for i in range(len(S)//2):
        for j in range(len(S)//2,len(S)):
             reh=reh+np.abs(S[i,j])**2
    # print(np.shape(S)[0]//2,ree,reh,ree+reh,2-ree+reh)
    return (np.shape(S)[0]//2-ree+reh)

def nonlocal_cond(S):
    ree=0
    reh=0
    for i in range(len(S)//2):
        for j in range(len(S)//2):
            ree=ree+np.abs(S[i,j])**2
    for i in range(len(S)//2):
        for j in range(len(S)//2,len(S)):
             reh=reh+np.abs(S[i,j])**2
    # print(np.shape(S)[0]//2,ree,reh)
    return ree,reh

def cal_local_conds(sys,omega):
    smatrix = kwant.smatrix(sys,energy=omega,check_hermiticity=True)
    cond=local_cond(np.array(smatrix.submatrix(0,0)))
    return cond

def cal_nonlocal_conds(sys,omega):
    smatrix = kwant.smatrix(sys,energy=omega,check_hermiticity=True)
    cond=nonlocal_cond(np.array(smatrix.submatrix(0,1)))
    return cond


B = 0.4
mu = 0
omega = 0
system = make_sys(B,omega,mu).finalized()


band_lead = kwant.physics.Bands(system.leads[0])
momenta = np.linspace(-pi, pi, 2001)
eng_lead = [band_lead(k) for k in momenta]


# def main():

#     t1 = time.time()
#     bias = np.linspace(-0.1,0.1,51)
#     Conds_LL = np.zeros(len(bias))    

#     for i in range(len(bias)):
#         omega = bias[i]
#         system = make_sys(B,omega,mu).finalized()
        
#         #GLL
#         Conds_LL[i] = cal_local_conds(system,omega)



#     #lead band
#     plt.figure(figsize=(10,4))
#     plt.subplot(1,2,1)
#     plt.plot(momenta, eng_lead,'b')
#     plt.xlabel('k')
#     plt.ylabel('engs')
#     plt.title('lead_bands')

#     #GLL
#     plt.subplot(1,2,2)
#     plt.plot(bias, Conds_LL/2)  
#     plt.xlabel(r"V_bias")
#     plt.ylabel("conductance [2e^2/h]")
#     plt.title(f'B={B}')
#     plt.show()


#     t2=time.time()
#     print(t2-t1)
    
# if __name__ == '__main__':
#     main()

# nonlocal

In [50]:
B = 0.
bias = np.linspace(-0.1,0.1,201)
Tee = np.zeros(len(bias))    
Teh = np.zeros(len(bias))    

for i in range(len(bias)):
    omega = bias[i]
    system = make_sys(B,omega,mu).finalized()
    
    #GLR
    Tee[i],Teh[i] = cal_nonlocal_conds(system,omega)


'Tee'
plt.figure(figsize=(10,4))
plt.subplot(1,2,1)
plt.plot(bias, Tee)
plt.xlabel('bias')
plt.title('Tee')

'Teh'
plt.subplot(1,2,2)
plt.plot(bias, Teh)  
plt.xlabel('bias')
plt.title('Teh')
plt.show()


plt.figure()
plt.plot(bias, Tee-Teh)
plt.title('GLR')
plt.show()
